In [7]:
#0
import pandas
from pandas import *
data2 = read_csv("/Users/ethanschultz/Documents/GSB 544/Data Files/avocado-updated-2020.csv")
#1
data2.rename(columns = {"4046":"small_hass"}, inplace = True)
data2.rename(columns = {"4225":"large_hass"}, inplace = True)
data2.rename(columns = {"4770":"extra_large_hass"}, inplace = True)
data2

,date,average_price,total_volume,small_hass,large_hass,extra_large_hass,total_bags,small_bags,large_bags,xlarge_bags,type,year,geography
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.00,conventional,2015,Albany
1,2015-01-04,1.79,1373.95,57.42,153.88,0.00,1162.65,1162.65,0.00,0.00,organic,2015,Albany
2,2015-01-04,1.00,435021.49,364302.39,23821.16,82.15,46815.79,16707.15,30108.64,0.00,conventional,2015,Atlanta
3,2015-01-04,1.76,3846.69,1500.15,938.35,0.00,1408.19,1071.35,336.84,0.00,organic,2015,Atlanta
4,2015-01-04,1.08,788025.06,53987.31,552906.04,39995.03,141136.68,137146.07,3990.61,0.00,conventional,2015,Baltimore/Washington
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33040,2020-11-29,1.47,1583056.27,67544.48,97996.46,2617.17,1414878.10,906711.52,480191.83,27974.75,organic,2020,Total U.S.
33041,2020-11-29,0.91,5811114.22,1352877.53,589061.83,19741.90,3790665.29,2197611.02,1531530.14,61524.13,conventional,2020,West
33042,2020-11-29,1.48,289961.27,13273.75,19341.09,636.51,256709.92,122606.21,134103.71,0.00,organic,2020,West
33043,2020-11-29,0.67,822818.75,234688.01,80205.15,10543.63,497381.96,285764.11,210808.02,809.83,conventional,2020,West Tex/New Mexico


#2
This data set has information about avocados from various years and dates within those years. It containts variables like price, the dtae of the observation, volume, the size of the avocados, the size of bags of avocados, whether or not the avocados were organic or not, and the geography of where the avocados came from.

In [11]:
geographys = data2['geography'].unique()
geographys
mapping_dict = {
    'Albany': 'Northeast', 'Baltimore/Washington': 'Northeast', 'Buffalo/Rochester': 'Northeast',
    'Hartford/Springfield': 'Northeast', 'New York': 'Northeast', 'Northern New England': 'Northeast',
    'Northeast': 'Northeast', 'Philadelphia': 'Northeast', 'Pittsburgh': 'Northeast', 'Syracuse': 'Northeast',
    'Atlanta': 'Southeast', 'Charlotte': 'Southeast', 'Jacksonville': 'Southeast',
    'Miami/Ft. Lauderdale': 'Southeast', 'Nashville': 'Southeast', 'Orlando': 'Southeast',
    'Raleigh/Greensboro': 'Southeast', 'Richmond/Norfolk': 'Southeast', 'Roanoke': 'Southeast',
    'South Carolina': 'Southeast', 'Southeast': 'Southeast', 'Tampa': 'Southeast',
    'Chicago': 'Midwest', 'Cincinnati/Dayton': 'Midwest', 'Columbus': 'Midwest', 'Detroit': 'Midwest',
    'Grand Rapids': 'Midwest', 'Great Lakes': 'Midwest', 'Indianapolis': 'Midwest',
    'Louisville': 'Midwest', 'Plains': 'Midwest', 'St. Louis': 'Midwest',
    'Dallas/Ft. Worth': 'Southwest & South Central', 'Houston': 'Southwest & South Central',
    'Las Vegas': 'Southwest & South Central', 'Midsouth': 'Southwest & South Central',
    'New Orleans/Mobile': 'Southwest & South Central', 'Phoenix/Tucson': 'Southwest & South Central',
    'South Central': 'Southwest & South Central', 'West Tex/New Mexico': 'Southwest & South Central',
    'Boise': 'West', 'California': 'West', 'Denver': 'West', 'Los Angeles': 'West',
    'Sacramento': 'West', 'San Diego': 'West', 'San Francisco': 'West',
    'Seattle': 'West', 'Spokane': 'West', 'Portland': 'West', 'West': 'West'
}

# Create new 'regions' column using .map()
data2['regions'] = data2['geography'].map(mapping_dict).fillna('Other')
data2


,date,average_price,total_volume,small_hass,large_hass,extra_large_hass,total_bags,small_bags,large_bags,xlarge_bags,type,year,geography,regions
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.00,conventional,2015,Albany,Northeast
1,2015-01-04,1.79,1373.95,57.42,153.88,0.00,1162.65,1162.65,0.00,0.00,organic,2015,Albany,Northeast
2,2015-01-04,1.00,435021.49,364302.39,23821.16,82.15,46815.79,16707.15,30108.64,0.00,conventional,2015,Atlanta,Southeast
3,2015-01-04,1.76,3846.69,1500.15,938.35,0.00,1408.19,1071.35,336.84,0.00,organic,2015,Atlanta,Southeast
4,2015-01-04,1.08,788025.06,53987.31,552906.04,39995.03,141136.68,137146.07,3990.61,0.00,conventional,2015,Baltimore/Washington,Northeast
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33040,2020-11-29,1.47,1583056.27,67544.48,97996.46,2617.17,1414878.10,906711.52,480191.83,27974.75,organic,2020,Total U.S.,Other
33041,2020-11-29,0.91,5811114.22,1352877.53,589061.83,19741.90,3790665.29,2197611.02,1531530.14,61524.13,conventional,2020,West,West
33042,2020-11-29,1.48,289961.27,13273.75,19341.09,636.51,256709.92,122606.21,134103.71,0.00,organic,2020,West,West
33043,2020-11-29,0.67,822818.75,234688.01,80205.15,10543.63,497381.96,285764.11,210808.02,809.83,conventional,2020,West Tex/New Mexico,Southwest & South Central


In [13]:
#3
data2.groupby('regions').sum('small_hass')

,average_price,total_volume,small_hass,large_hass,extra_large_hass,total_bags,small_bags,large_bags,xlarge_bags,year
regions,,,,,,,,,,
Midwest,8158.05,2.635496e+09,6.302978e+08,8.031268e+08,1.633969e+08,1.037191e+09,7.763920e+08,2.224286e+08,38370772.24,12346860
Northeast,9204.75,2.917646e+09,1.753555e+08,1.792215e+09,2.228612e+07,9.277672e+08,7.383827e+08,1.812659e+08,8118593.48,12346860
Other,2598.52,1.239188e+10,3.874749e+09,3.631053e+09,2.683674e+08,4.616657e+09,3.182302e+09,1.337010e+09,97344646.14,3704058
Southeast,9902.35,2.705103e+09,1.278921e+09,3.130368e+08,2.055340e+07,1.092591e+09,6.957548e+08,3.836494e+08,13187048.95,14816232
Southwest & South Central,5921.11,4.897086e+09,2.026573e+09,1.030316e+09,9.309349e+07,1.747009e+09,1.191409e+09,5.353734e+08,20226675.99,9871439
West,9815.36,6.453557e+09,2.006627e+09,1.675229e+09,1.421935e+08,2.629419e+09,1.683553e+09,8.672606e+08,78605972.54,13581546
